In [ ]:
!lscpu

In [ ]:
!nvidia-smi

# **Preparation**

In [ ]:
!pip install adapter-transformers transformers datasets evaluate

In [ ]:
!gdown 'https://drive.google.com/uc?id=1KWokWHSR9i5QYzaY1PGmB7qAzsj2SuOk'
!unzip lexical_datasets

# **Script**

In [ ]:
!python lrc_train_evaluate_adapters.py \
	--train_templates "' <W1> ' <SEP> ' <W2> '" \
	--model  "roberta-base" \
  --use_adapters True \
	--nepochs 10 \
	--dir_output_results "results/" \
	--batch_size 32 \
	--nrepetitions 1 \
	--dataset "EVALution" \
	--date `date "+%D-%H:%M:%S"` \
	--train_file "lexical_datasets/EVALution/train.tsv" \
	--test_file "lexical_datasets/EVALution/test.tsv" \
	--val_file "lexical_datasets/EVALution/val.tsv"

# **Loading the tokenizer**

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig, RobertaModelWithHeads, AdapterConfig, TrainingArguments, Trainer, AdapterTrainer
from sklearn.metrics import classification_report
from google.colab import files
from collections import Counter

model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)

# **Preparing the data**

In [ ]:
import re

# templates to verbalize the words in a relation
template1 = "' <W1> ' <SEP> ' <W2> '"
template2 = " <W1> <SEP> <W2> "

# function to verbalize the words in a relation
def verbalize_words(row, template, tokenizer):
    w1 = str(row['source'])
    w2 = str(row['target'])
    sentence = re.sub("<W1>", w1, template)
    sentence = re.sub("<W2>", w2, sentence)
    sentence = re.sub("<SEP>", tokenizer.sep_token, sentence)
    return {'verb':sentence}

In [ ]:
def preprocess_function(rows, tokenizer):
    """ tokenize the column 'verb' of the rows"""
    inputs = tokenizer(rows['verb'], padding='max_length', max_length=64, return_tensors="pt")
    return inputs

In [ ]:
from datasets import load_dataset

def load_data(data_files):
    # load the train/val/test datasets
    all_datasets = load_dataset('csv',
                            data_files=data_files,
                            sep = '\t',
                            header=None,
                            names=['source', 'target', 'rel'],
                            keep_default_na=False)

    # create the column 'labels', copy of column 'rel'
    all_datasets = all_datasets.map(lambda x: {'labels':x['rel']})

    # trasform column 'labels' to a integer with a label id. Needed to fine-tune the model
    all_datasets = all_datasets.class_encode_column('labels')

    # add a column to the train/val/test datasets with the verbalization
    all_datasets = all_datasets.map(verbalize_words, fn_kwargs={'template':template1, 'tokenizer':tokenizer})

    # prepare datasets for the LM. We need input_ids, attention_mask and labels (the correct ones)
    encoded_datasets = all_datasets.map(preprocess_function, batched=True, batch_size=None,fn_kwargs={'tokenizer':tokenizer})

    # remove all columns except those needed to train/evalauate the model
    # this step is not necessary. We can leave the columns.
    encoded_datasets = encoded_datasets.remove_columns(['source', 'target', 'rel', 'verb'])

    return all_datasets, encoded_datasets

In [ ]:
import evaluate
import numpy as np

# we will get the best model based on the macro f1 score over the val dataset
# download the metric from huggingface using the evaluate package
metric_name = "f1"
metric = evaluate.load(metric_name)

def compute_metrics(eval_pred):
    '''
    Compute metrics for a Trainer.

    Args:
     eval_pred: object of type transformers.EvalPrediction. It is a tuple with
     predictions (logits) and real labels.

    Returns:
     A dictionary of metrics {'name_metric1':value1,...}
    '''
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return metric.compute(predictions=predictions, references=labels, average='macro')

In [ ]:
def save_adapter(model, adapter_name):
    model.save_adapter(adapter_name, adapter_name)
    !zip -r {adapter_name}.zip {adapter_name}

In [ ]:
def results_to_file(dataset, real_rels, pred_rels, file):
  with open(file, 'w') as f:
        f.write(f'Training Time: minutes\n\n')
        f.write(str(report_dict))
        f.write('\n\n')
        for i, row in enumerate(dataset):
          source = row["source"]
          target = row["target"]
          rel = row["rel"]
          if rel != real_rels[i]:
            raise ValueError
          pred = pred_rels[i]
          f.write(f"{source} {target} {rel} {pred}\n")
  files.download(file)

# **Adapters & Fine-Tuning training with EVALution**

In [ ]:
all_datasets_EVALution, encoded_datasets_EVALution = load_data({'train':'lexical_datasets/EVALution/train.tsv', 'val':'lexical_datasets/EVALution/val.tsv', 'test':'lexical_datasets/EVALution/test.tsv'})
print(all_datasets_EVALution)
print(encoded_datasets_EVALution)

In [ ]:
etiquetas_train = all_datasets_EVALution["train"]["rel"]
etiquetas_test = all_datasets_EVALution["test"]["rel"]
etiquetas_val = all_datasets_EVALution["val"]["rel"]

conteo_etiquetas_train = Counter(etiquetas_train)
conteo_etiquetas_test = Counter(etiquetas_test)
conteo_etiquetas_val = Counter(etiquetas_val)

etiquetas = set(etiquetas_train)

for etiqueta in etiquetas:
    conteo_train = conteo_etiquetas_train.get(etiqueta, 0)
    conteo_test = conteo_etiquetas_test.get(etiqueta, 0)
    conteo_val = conteo_etiquetas_val.get(etiqueta, 0)
    print(f"{etiqueta}: {conteo_train}, {conteo_val}, {conteo_test}")

In [ ]:
print(all_datasets_EVALution['train'].features['labels'].names)
num_labels = all_datasets_EVALution['train'].features['labels'].num_classes
print(f"Number of labels: {num_labels}")

In [ ]:
print(conteo_etiquetas_train.get('MadeOf', 0) + conteo_etiquetas_train.get('PartOf', 0))
print(conteo_etiquetas_test.get('MadeOf', 0) + conteo_etiquetas_test.get('PartOf', 0))
print(conteo_etiquetas_val.get('MadeOf', 0) + conteo_etiquetas_val.get('PartOf', 0))

## **Training the adapter**

In [ ]:
config = RobertaConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
)
model = RobertaModelWithHeads.from_pretrained(
    model_name,
    config=config,
)

In [ ]:
labels = all_datasets_EVALution['train']['labels']
rel = all_datasets_EVALution['train']['rel']
unique_pairs = set(zip(labels, rel))
id2label = dict(unique_pairs)

print(id2label)

In [ ]:
adapter_name = "adapter-EVALution"

In [ ]:
# Add a new adapter
model.add_adapter(adapter_name)
# Add a matching classification head
model.add_classification_head(
    adapter_name,
    num_labels=num_labels,
    id2label=id2label
  )
# Activate the adapter
model.train_adapter(adapter_name)

In [ ]:
batch_size = 32
total_epochs = 10

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=total_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=2*batch_size,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    report_to='all',
    load_best_model_at_end=True,     #load the best model at the end of training,
    metric_for_best_model=metric_name,   #use metric_name for validation
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,
    optim='adamw_torch'
)

In [ ]:
trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=encoded_datasets_EVALution['train'],
    eval_dataset=encoded_datasets_EVALution['val'],
    tokenizer=tokenizer, #it is needed the tokenizer that encoded the data for batching
    compute_metrics=compute_metrics #to compute metric of the model in val dataset
)

In [ ]:
trainer.train()

In [ ]:
preds = trainer.predict(test_dataset=encoded_datasets_EVALution['test'])
pred_labels = np.argmax(preds.predictions, axis = 1)

In [ ]:
real_rels = encoded_datasets_EVALution['test'].features['labels'].int2str(encoded_datasets_EVALution['test']['labels'])
pred_rels = encoded_datasets_EVALution['test'].features['labels'].int2str(pred_labels)
report = classification_report(real_rels, pred_rels, digits=3)
report_dict = classification_report(real_rels, pred_rels, digits=3, output_dict=True)
print(report)
print(report_dict)

In [ ]:
dataset = all_datasets_EVALution['test']
file = 'results_iteration_4.txt'
results_to_file(dataset, real_rels, pred_rels, file)

## **Fine-tunning the model**



In [ ]:
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

In [ ]:
batch_size = 32
total_epochs = 10

args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=total_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=2*batch_size,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    report_to='all',
    load_best_model_at_end=True,     #load the best model at the end of training,
    metric_for_best_model=metric_name,   #use metric_name for validation
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,
    optim='adamw_torch'
)

In [ ]:
trainer = Trainer(
    model=model, #model to train
    args=args,  #arguments to train
    train_dataset=encoded_datasets_EVALution['train'],
    eval_dataset=encoded_datasets_EVALution['val'],
    tokenizer=tokenizer, #it is needed the tokenizer that encoded the data for batching
    compute_metrics=compute_metrics #to compute metric of the model in val dataset
)

In [ ]:
trainer.train()

In [ ]:
preds = trainer.predict(test_dataset=encoded_datasets_EVALution['test'])
pred_labels = np.argmax(preds.predictions, axis = 1)

In [ ]:
real_rels = encoded_datasets_EVALution['test'].features['labels'].int2str(encoded_datasets_EVALution['test']['labels'])
pred_rels = encoded_datasets_EVALution['test'].features['labels'].int2str(pred_labels)
report = classification_report(real_rels, pred_rels, digits=3)
report_dict = classification_report(real_rels, pred_rels, digits=3, output_dict=True)
print(report)
print(report_dict)

In [ ]:
dataset = all_datasets_EVALution['test']
file = 'results_iteration_4.txt'
results_to_file(dataset, real_rels, pred_rels, file)

# **Adapters & Fine-Tuning training with ROOT09**

In [ ]:
all_datasets_ROOT09, encoded_datasets_ROOT09 = load_data({'train':'lexical_datasets/ROOT09/train.tsv', 'val':'lexical_datasets/ROOT09/val.tsv', 'test':'lexical_datasets/ROOT09/test.tsv'})
print(all_datasets_ROOT09)
print(encoded_datasets_ROOT09)

In [ ]:
etiquetas_train = all_datasets_ROOT09["train"]["rel"]
etiquetas_test = all_datasets_ROOT09["test"]["rel"]
etiquetas_val = all_datasets_ROOT09["val"]["rel"]

conteo_etiquetas_train = Counter(etiquetas_train)
conteo_etiquetas_test = Counter(etiquetas_test)
conteo_etiquetas_val = Counter(etiquetas_val)

etiquetas = set(etiquetas_train)

for etiqueta in etiquetas:
    conteo_train = conteo_etiquetas_train.get(etiqueta, 0)
    conteo_test = conteo_etiquetas_test.get(etiqueta, 0)
    conteo_val = conteo_etiquetas_val.get(etiqueta, 0)
    print(f"{etiqueta}: {conteo_train}, {conteo_val}, {conteo_test}")

In [ ]:
print(all_datasets_ROOT09['train'].features['labels'].names)
num_labels = all_datasets_ROOT09['train'].features['labels'].num_classes
print(f"Number of labels: {num_labels}")

## **Training the adapter**

In [ ]:
config = RobertaConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
)
model = RobertaModelWithHeads.from_pretrained(
    model_name,
    config=config,
)

In [ ]:
labels = all_datasets_ROOT09['train']['labels']
rel = all_datasets_ROOT09['train']['rel']
unique_pairs = set(zip(labels, rel))
id2label = dict(unique_pairs)

print(id2label)

In [ ]:
adapter_name = "adapter-ROOT09"

In [ ]:
# Add a new adapter
model.add_adapter(adapter_name)
# Add a matching classification head
model.add_classification_head(
    adapter_name,
    num_labels=num_labels,
    id2label=id2label
  )
# Activate the adapter
model.train_adapter(adapter_name)

In [ ]:
batch_size = 32
total_epochs = 10

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=total_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=2*batch_size,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    report_to='all',
    load_best_model_at_end=True,     #load the best model at the end of training,
    metric_for_best_model=metric_name,   #use metric_name for validation
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,
    optim='adamw_torch'
)

In [ ]:
trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=encoded_datasets_ROOT09['train'],
    eval_dataset=encoded_datasets_ROOT09['val'],
    tokenizer=tokenizer, #it is needed the tokenizer that encoded the data for batching
    compute_metrics=compute_metrics #to compute metric of the model in val dataset
)

In [ ]:
trainer.train()

In [ ]:
preds = trainer.predict(test_dataset=encoded_datasets_ROOT09['test'])
pred_labels = np.argmax(preds.predictions, axis = 1)

In [ ]:
real_rels = encoded_datasets_ROOT09['test'].features['labels'].int2str(encoded_datasets_ROOT09['test']['labels'])
pred_rels = encoded_datasets_ROOT09['test'].features['labels'].int2str(pred_labels)
report = classification_report(real_rels, pred_rels, digits=3)
report_dict = classification_report(real_rels, pred_rels, digits=3, output_dict=True)
print(report)
print(report_dict)

In [ ]:
dataset = all_datasets_ROOT09['test']
file = 'results_iteration_4.txt'
results_to_file(dataset, real_rels, pred_rels, file)

## **Fine-tunning the model**

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

In [ ]:
batch_size = 32
total_epochs = 10

args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=total_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=2*batch_size,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    report_to='all',
    load_best_model_at_end=True,     #load the best model at the end of training,
    metric_for_best_model=metric_name,   #use metric_name for validation
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,
    optim='adamw_torch'
)

In [ ]:
trainer = Trainer(
    model=model, #model to train
    args=args,  #arguments to train
    train_dataset=encoded_datasets_ROOT09['train'],
    eval_dataset=encoded_datasets_ROOT09['val'],
    tokenizer=tokenizer, #it is needed the tokenizer that encoded the data for batching
    compute_metrics=compute_metrics #to compute metric of the model in val dataset
)

In [ ]:
trainer.train()

In [ ]:
preds = trainer.predict(test_dataset=encoded_datasets_ROOT09['test'])
pred_labels = np.argmax(preds.predictions, axis = 1)

In [ ]:
real_rels = encoded_datasets_ROOT09['test'].features['labels'].int2str(encoded_datasets_ROOT09['test']['labels'])
pred_rels = encoded_datasets_ROOT09['test'].features['labels'].int2str(pred_labels)
report = classification_report(real_rels, pred_rels, digits=3)
report_dict = classification_report(real_rels, pred_rels, digits=3, output_dict=True)
print(report)
print(report_dict)

In [ ]:
dataset = all_datasets_ROOT09['test']
file = 'results_iteration_4.txt'
results_to_file(dataset, real_rels, pred_rels, file)

# **Adapters & Fine-Tuning training with CogALex-V**

In [ ]:
all_datasets_CogALexV, encoded_datasets_CogALexV = load_data({'train':'lexical_datasets/CogALexV/train.tsv', 'test':'lexical_datasets/CogALexV/test.tsv'})
print(all_datasets_CogALexV)
print(encoded_datasets_CogALexV)

In [ ]:
etiquetas_train = all_datasets_CogALexV["train"]["rel"]
etiquetas_test = all_datasets_CogALexV["test"]["rel"]

conteo_etiquetas_train = Counter(etiquetas_train)
conteo_etiquetas_test = Counter(etiquetas_test)

etiquetas = set(etiquetas_train)

for etiqueta in etiquetas:
    conteo_train = conteo_etiquetas_train.get(etiqueta, 0)
    conteo_test = conteo_etiquetas_test.get(etiqueta, 0)
    print(f"{etiqueta}: {conteo_train}, {conteo_test}")

In [ ]:
print(all_datasets_CogALexV['train'].features['labels'].names)
num_labels = all_datasets_CogALexV['train'].features['labels'].num_classes
print(f"Number of labels: {num_labels}")

## **Training the adapter**

In [ ]:
config = RobertaConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
)
model = RobertaModelWithHeads.from_pretrained(
    model_name,
    config=config,
)

In [ ]:
labels = all_datasets_CogALexV['train']['labels']
rel = all_datasets_CogALexV['train']['rel']
unique_pairs = set(zip(labels, rel))
id2label = dict(unique_pairs)

print(id2label)

In [ ]:
adapter_name = "adapter-CogALexV"

In [ ]:
# Add a new adapter
model.add_adapter(adapter_name)
# Add a matching classification head
model.add_classification_head(
    adapter_name,
    num_labels=num_labels,
    id2label=id2label
  )
# Activate the adapter
model.train_adapter(adapter_name)

In [ ]:
batch_size = 32
total_epochs = 10

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=total_epochs,
    per_device_train_batch_size=batch_size,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    report_to='all',
    save_strategy='epoch',
    save_total_limit=1,
    optim='adamw_torch'
)

In [ ]:
trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=encoded_datasets_CogALexV['train'],
    tokenizer=tokenizer, #it is needed the tokenizer that encoded the data for batching
)

In [ ]:
trainer.train()

In [ ]:
preds = trainer.predict(test_dataset=encoded_datasets_CogALexV['test'])
pred_labels = np.argmax(preds.predictions, axis = 1)

In [ ]:
real_rels = encoded_datasets_CogALexV['test'].features['labels'].int2str(encoded_datasets_CogALexV['test']['labels'])
pred_rels = encoded_datasets_CogALexV['test'].features['labels'].int2str(pred_labels)
report = classification_report(real_rels, pred_rels, digits=3)
report_dict = classification_report(real_rels, pred_rels, digits=3, output_dict=True)
print(report)
print(report_dict)

In [ ]:
dataset = all_datasets_CogALexV['test']
file = 'results_iteration_4.txt'
results_to_file(dataset, real_rels, pred_rels, file)

## **Fine-tunning the model**

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

In [ ]:
batch_size = 32
total_epochs = 10

args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=total_epochs,
    per_device_train_batch_size=batch_size,
    logging_steps=100,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    report_to='all',
    save_strategy='epoch',
    save_total_limit=1,
    optim='adamw_torch'
)

In [ ]:
trainer = Trainer(
    model=model, #model to train
    args=args,  #arguments to train
    train_dataset=encoded_datasets_CogALexV['train'],
    tokenizer=tokenizer, #it is needed the tokenizer that encoded the data for batching
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
preds = trainer.predict(test_dataset=encoded_datasets_CogALexV['test'])
pred_labels = np.argmax(preds.predictions, axis = 1)

In [ ]:
real_rels = encoded_datasets_CogALexV['test'].features['labels'].int2str(encoded_datasets_CogALexV['test']['labels'])
pred_rels = encoded_datasets_CogALexV['test'].features['labels'].int2str(pred_labels)
report = classification_report(real_rels, pred_rels, digits=3)
report_dict = classification_report(real_rels, pred_rels, digits=3, output_dict=True)
print(report)
print(report_dict)

In [ ]:
dataset = all_datasets_CogALexV['test']
file = 'results_iteration_4.txt'
results_to_file(dataset, real_rels, pred_rels, file)

# **Analysis**

## **Extraction of misclassified relations in all iterations & Overlap errors between adapters and the model**

In [ ]:
# Función para procesar un archivo y encontrar las relaciones mal predichas
def procesar_archivo(nombre_archivo):
    relaciones_mal_predichas = []
    with open(nombre_archivo, 'r') as file:
        lines = file.readlines()[4:]  # Eliminar las primeras líneas

        for line in lines:
            line = line.strip()
            words = line.split()

            source = words[0]
            target = words[1]
            real_clas = words[2]
            pred_clas = words[3]

            if real_clas != pred_clas:
                relaciones_mal_predichas.append((source, target, real_clas, pred_clas))

    return relaciones_mal_predichas

In [ ]:
def key_function(item):
    return item[0].lower(), item[1].lower(), item[2], item[3]

archivos = ['results_iteration_0.txt', 'results_iteration_1.txt', 'results_iteration_2.txt',
         'results_iteration_3.txt', 'results_iteration_4.txt']
output = 'misclassified_relations_rb_CogALexV_model.txt'
relaciones_mal_predichas_por_archivo = []

for archivo in archivos:
    relaciones_mal_predichas = procesar_archivo(archivo)
    relaciones_mal_predichas_por_archivo.append(relaciones_mal_predichas)

for i, relaciones_mal_predichas in enumerate(relaciones_mal_predichas_por_archivo):
    print(f"Longitud de relaciones_mal_predichas_por_archivo[{i}]: {len(relaciones_mal_predichas)}")

relaciones_mal_predichas_comunes = set.intersection(*map(set, relaciones_mal_predichas_por_archivo))
print(f"Intersección de relaciones_mal_predichas_por_archivo: {len(relaciones_mal_predichas_comunes)}")

relaciones_mal_predichas_comunes = sorted(relaciones_mal_predichas_comunes, key=key_function)
with open(output, "w") as file:
    for relacion in relaciones_mal_predichas_comunes:
        file.write(f"{relacion[0]} {relacion[1]} {relacion[2]} {relacion[3]}\n")

In [ ]:
# Función para procesar un archivo y encontrar las relaciones mal predichas
def procesar_archivo(nombre_archivo):
    relaciones_mal_predichas = []
    with open(nombre_archivo, 'r') as file:
        lines = file.readlines()

        for line in lines:
            line = line.strip()
            words = line.split()

            source = words[0]
            target = words[1]
            real_clas = words[2]
            pred_clas = words[3]

            if real_clas != pred_clas:
                relaciones_mal_predichas.append((source, target, real_clas, pred_clas))

    return relaciones_mal_predichas

In [ ]:
def key_function(item):
    return item[0].lower(), item[1].lower(), item[2], item[3]

adapter = 'misclassified_relations_rb_CogALexV_adapters.txt'
model = 'misclassified_relations_rb_CogALexV_model.txt'
output = 'comparison_errors_rb_CogALexV.txt'

relaciones_mal_predichas_adapter = procesar_archivo(adapter)
relaciones_mal_predichas_model = procesar_archivo(model)

print(f"Longitud de relaciones_mal_predichas_adapter: {len(relaciones_mal_predichas_adapter)}")
print(f"Longitud de relaciones_mal_predichas_model: {len(relaciones_mal_predichas_model)}")

set_adapter = set(relaciones_mal_predichas_adapter)
set_model = set(relaciones_mal_predichas_model)

interseccion = sorted(set_adapter.intersection(set_model), key=key_function)
diferencia_adapter = sorted(set_adapter - set_model, key=key_function)
diferencia_model = sorted(set_model - set_adapter, key=key_function)

with open(output, "w") as file:
    file.write(f"Mal clasificadas en ambos casos: {len(interseccion)}\n\n")
    for relacion in interseccion:
        file.write(f"{relacion[0]} {relacion[1]} {relacion[2]} {relacion[3]}\n")

    file.write(f"\nMal clasificadas con adapter y bien solo con modelo: {len(diferencia_adapter)}\n\n")
    for relacion in diferencia_adapter:
        file.write(f"{relacion[0]} {relacion[1]} {relacion[2]} {relacion[3]}\n")

    file.write(f"\nMal clasificadas con solo modelo y bien con adapter: {len(diferencia_model)}\n\n")
    for relacion in diferencia_model:
        file.write(f"{relacion[0]} {relacion[1]} {relacion[2]} {relacion[3]}\n")

## **Extraction of classified relations in all iterations**

In [ ]:
# Función para procesar un archivo y encontrar las relaciones bien predichas
def procesar_archivo(nombre_archivo):
    relaciones_bien_predichas = []
    with open(nombre_archivo, 'r') as file:
        lines = file.readlines()[4:]  # Eliminar las primeras líneas

        for line in lines:
            line = line.strip()
            words = line.split()

            source = words[0]
            target = words[1]
            real_clas = words[2]
            pred_clas = words[3]

            if real_clas == pred_clas:
                relaciones_bien_predichas.append((source, target, real_clas, pred_clas))

    return relaciones_bien_predichas

In [ ]:
def key_function(item):
    return item[0].lower(), item[1].lower(), item[2], item[3]

archivos = ['results_iteration_0.txt', 'results_iteration_1.txt', 'results_iteration_2.txt',
         'results_iteration_3.txt', 'results_iteration_4.txt']
output = 'classified_relations_rl_EVALution_adapters.txt'
relaciones_bien_predichas_por_archivo = []

for archivo in archivos:
    relaciones_bien_predichas = procesar_archivo(archivo)
    relaciones_bien_predichas_por_archivo.append(relaciones_bien_predichas)

for i, relaciones_bien_predichas in enumerate(relaciones_bien_predichas_por_archivo):
    print(f"Longitud de relaciones_bien_predichas_por_archivo[{i}]: {len(relaciones_bien_predichas)}")

relaciones_bien_predichas_comunes = set.intersection(*map(set, relaciones_bien_predichas_por_archivo))
print(f"Intersección de relaciones_bien_predichas_por_archivo: {len(relaciones_bien_predichas_comunes)}")

relaciones_bien_predichas_comunes = sorted(relaciones_bien_predichas_comunes, key=key_function)
with open(output, "w") as file:
    for relacion in relaciones_bien_predichas_comunes:
        file.write(f"{relacion[0]} {relacion[1]} {relacion[2]} {relacion[3]}\n")

In [ ]:
def get_num_good_pred(nombre_archivo):
    relaciones_acertadas = {}
    num_pairs = 0

    with open(nombre_archivo, 'r') as file:
        lines = file.readlines()

        for line in lines:
            line = line.strip()
            words = line.split()

            real_class = words[2]
            pred_class = words[3]

            if real_class != pred_class:
               print("Wrong file!")

            if real_class in relaciones_acertadas:
              relaciones_acertadas[real_class] += 1
            else:
              relaciones_acertadas[real_class] = 1

            num_pairs = num_pairs + 1

    return relaciones_acertadas, num_pairs

In [ ]:
relaciones_acertadas, num_pairs = get_num_good_pred('classified_relations_rl_CogALexV_model.txt')
print(relaciones_acertadas)
print(f"Total aciertos: {num_pairs}")

## **Confusion matrix for misclassified relations in all iterations with both adapters and the model**

In [ ]:
def get_true_pred_labels(nombre_archivo):
    true_labels = []
    pred_labels = []
    relaciones_falladas = {}

    with open(nombre_archivo, 'r') as file:
        lines = file.readlines()

        for line in lines:
            line = line.strip()
            words = line.split()

            real_class = words[2]
            pred_class = words[3]

            true_labels.append(real_class)
            pred_labels.append(pred_class)

            if real_class in relaciones_falladas:
              relaciones_falladas[real_class] += 1
            else:
              relaciones_falladas[real_class] = 1

    return true_labels, pred_labels, relaciones_falladas

In [ ]:
y_true, y_pred, relaciones_falladas = get_true_pred_labels('comparison_errors_rl_EVALution.txt')
print(relaciones_falladas)
print(len(y_true), len(y_pred))

In [ ]:
diccionario = {}
for true, pred in zip(y_true, y_pred):
  if true not in diccionario:
    diccionario[true] = [(pred, 1)]
  else:
    indice = next((i for i, tupla in enumerate(diccionario[true]) if tupla[0] == pred), None)
    if indice is None:
      diccionario[true].append((pred, 1))
    else:
      tupla = diccionario[true][indice]
      diccionario[true][indice] = (tupla[0], tupla[1] + 1)

print(diccionario)

In [ ]:
porcentajes_fallos_respecto_resto = {}
for clave, valor in diccionario.items():
  porcentajes_fallos_respecto_resto[clave] = [(rel, (num/relaciones_falladas[clave])*100) for (rel, num) in valor]

print(porcentajes_fallos_respecto_resto)

In [ ]:
import numpy as np

etiquetas = np.unique(y_true) if len(np.unique(y_true)) > len(np.unique(y_pred)) else np.unique(y_pred)
print(etiquetas)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', xticklabels=etiquetas, yticklabels=etiquetas)
plt.xlabel('Predicciones')
plt.ylabel('Valores reales')
plt.title('Matriz de Confusión')
plt.show()

## **Average confusion matrix for misclassified relations**

In [ ]:
def get_labels(nombre_archivo):
    true_labels = []
    pred_labels = []

    with open(nombre_archivo, 'r') as file:
        lines = file.readlines()[4:]

        for line in lines:
            line = line.strip()
            words = line.split()

            real_class = words[2]
            pred_class = words[3]

            true_labels.append(real_class)
            pred_labels.append(pred_class)

    return true_labels, pred_labels

In [ ]:
y_true_0, y_pred_0 = get_labels('results_iteration_0.txt')
y_true_1, y_pred_1 = get_labels('results_iteration_1.txt')
y_true_2, y_pred_2 = get_labels('results_iteration_2.txt')
y_true_3, y_pred_3 = get_labels('results_iteration_3.txt')
y_true_4, y_pred_4 = get_labels('results_iteration_4.txt')

In [ ]:
import numpy as np

etiquetas = np.unique(y_true_0) if len(np.unique(y_true_0)) > len(np.unique(y_pred_0)) else np.unique(y_pred_0)
print(etiquetas)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

matriz_confusion_1 = confusion_matrix(y_true_0, y_pred_0)
matriz_confusion_2 = confusion_matrix(y_true_1, y_pred_1)
matriz_confusion_3 = confusion_matrix(y_true_2, y_pred_2)
matriz_confusion_4 = confusion_matrix(y_true_3, y_pred_3)
matriz_confusion_5 = confusion_matrix(y_true_4, y_pred_4)

suma_matriz_confusion = matriz_confusion_1 + matriz_confusion_2 + matriz_confusion_3 + matriz_confusion_4 + matriz_confusion_5

matriz_confusion_promedio = suma_matriz_confusion / 5

np.fill_diagonal(matriz_confusion_promedio, 0)

plt.figure(figsize=(8, 6))
sns.heatmap(matriz_confusion_promedio, annot=True, cmap='Blues', xticklabels=etiquetas, yticklabels=etiquetas)
plt.xlabel('Predicciones')
plt.ylabel('Valores reales')
plt.title('Matriz de Confusión Promedio')
plt.show()

In [ ]:
suma_filas = matriz_confusion_promedio.sum(axis=1)
matriz_confusion_porcentaje = (matriz_confusion_promedio / suma_filas[:, np.newaxis]) * 100

print(matriz_confusion_porcentaje)

## **Degree of overlap errors between adapters and the model**

In [ ]:
def get_rels(nombre_archivo):
    fallos = []

    with open(nombre_archivo, 'r') as file:
        lines = file.readlines()

        for line in lines:
            line = line.strip()
            words = line.split()

            source = words[0]
            target = words[1]
            real_class = words[2]
            pred_class = words[3]

            fallos.append((source, target, real_class))

    return fallos

In [ ]:
fallos_adapter = get_rels('misclassified_relations_rl_CogALexV_adapters.txt')
fallos_model = get_rels('misclassified_relations_rl_CogALexV_model.txt')

In [ ]:
print(len(fallos_adapter))
print(len(set(fallos_adapter)))

In [ ]:
print(len(fallos_model))
print(len(set(fallos_model)))

In [ ]:
fallos_adapter_set = set(fallos_adapter)
fallos_model_set = set(fallos_model)

interseccion = fallos_adapter_set.intersection(fallos_model_set)
union = fallos_adapter_set.union(fallos_model_set)

porcentaje_solape = (len(interseccion) / len(union)) * 100

print("Porcentaje de solape:", porcentaje_solape)

In [ ]:
print(len(interseccion))
print(len(union))

# **Adapter layer combinations**



## **CogALexV**

In [ ]:
all_datasets_CogALexV, encoded_datasets_CogALexV = load_data({'train':'lexical_datasets/CogALexV/train.tsv', 'test':'lexical_datasets/CogALexV/test.tsv'})
print(all_datasets_CogALexV)
print(encoded_datasets_CogALexV)

In [ ]:
print(all_datasets_CogALexV['train'].features['labels'].names)
num_labels = all_datasets_CogALexV['train'].features['labels'].num_classes
print(f"Number of labels: {num_labels}")

In [ ]:
config = RobertaConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
)
model = RobertaModelWithHeads.from_pretrained(
    model_name,
    config=config,
)

In [ ]:
labels = all_datasets_CogALexV['train']['labels']
rel = all_datasets_CogALexV['train']['rel']
unique_pairs = set(zip(labels, rel))
id2label = dict(unique_pairs)

print(id2label)

In [ ]:
adapter_name = "adapter-CogALexV"

In [ ]:
leave_out = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
print(leave_out)

In [ ]:
# PfeifferConfig + leave_out
adapter_config = AdapterConfig(
    original_ln_before = True,
    original_ln_after = True,
    residual_before_ln = True,
    adapter_residual_before_ln = False,
    ln_before = False,
    ln_after = False,
    mh_adapter = False,
    output_adapter = True,
    non_linearity = "relu",
    reduction_factor = 16,
    leave_out = leave_out
)

In [ ]:
# Add a new adapter
model.add_adapter(adapter_name, config=adapter_config)
# Add a matching classification head
model.add_classification_head(
    adapter_name,
    num_labels=num_labels,
    id2label=id2label
  )
# Activate the adapter
model.train_adapter(adapter_name)

In [ ]:
adapter_layers = model.get_adapter(adapter_name)
print(adapter_layers)

In [ ]:
batch_size = 32
total_epochs = 10

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=total_epochs,
    per_device_train_batch_size=batch_size,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    report_to='all',
    save_strategy='epoch',
    save_total_limit=1,
    optim='adamw_torch'
)

In [ ]:
trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=encoded_datasets_CogALexV['train'],
    tokenizer=tokenizer, #it is needed the tokenizer that encoded the data for batching
)

In [ ]:
trainer.train()

In [ ]:
preds = trainer.predict(test_dataset=encoded_datasets_CogALexV['test'])
pred_labels = np.argmax(preds.predictions, axis = 1)

In [ ]:
real_rels = encoded_datasets_CogALexV['test'].features['labels'].int2str(encoded_datasets_CogALexV['test']['labels'])
pred_rels = encoded_datasets_CogALexV['test'].features['labels'].int2str(pred_labels)
report = classification_report(real_rels, pred_rels, digits=3)
report_dict = classification_report(real_rels, pred_rels, digits=3, output_dict=True)
print(report)
print(report_dict)

In [ ]:
dataset = all_datasets_CogALexV['test']
file = 'rl_CogALexV_16-23_4.txt'
results_to_file(dataset, real_rels, pred_rels, file)

## **EVALution**

In [ ]:
all_datasets_EVALution, encoded_datasets_EVALution = load_data({'train':'lexical_datasets/EVALution/train.tsv', 'val':'lexical_datasets/EVALution/val.tsv', 'test':'lexical_datasets/EVALution/test.tsv'})
print(all_datasets_EVALution)
print(encoded_datasets_EVALution)

In [ ]:
print(all_datasets_EVALution['train'].features['labels'].names)
num_labels = all_datasets_EVALution['train'].features['labels'].num_classes
print(f"Number of labels: {num_labels}")

In [ ]:
config = RobertaConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
)
model = RobertaModelWithHeads.from_pretrained(
    model_name,
    config=config,
)

In [ ]:
labels = all_datasets_EVALution['train']['labels']
rel = all_datasets_EVALution['train']['rel']
unique_pairs = set(zip(labels, rel))
id2label = dict(unique_pairs)

print(id2label)

In [ ]:
adapter_name = "adapter-EVALution"

In [ ]:
leave_out = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
print(leave_out)

In [ ]:
# PfeifferConfig + leave_out
adapter_config = AdapterConfig(
    original_ln_before = True,
    original_ln_after = True,
    residual_before_ln = True,
    adapter_residual_before_ln = False,
    ln_before = False,
    ln_after = False,
    mh_adapter = False,
    output_adapter = True,
    non_linearity = "relu",
    reduction_factor = 16,
    leave_out = leave_out
)

In [ ]:
# Add a new adapter
model.add_adapter(adapter_name, config=adapter_config)
# Add a matching classification head
model.add_classification_head(
    adapter_name,
    num_labels=num_labels,
    id2label=id2label
  )
# Activate the adapter
model.train_adapter(adapter_name)

In [ ]:
adapter_layers = model.get_adapter(adapter_name)
print(adapter_layers)

In [ ]:
batch_size = 32
total_epochs = 10

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=total_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=2*batch_size,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    report_to='all',
    load_best_model_at_end=True,     #load the best model at the end of training,
    metric_for_best_model=metric_name,   #use metric_name for validation
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,
    optim='adamw_torch'
)

In [ ]:
trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=encoded_datasets_EVALution['train'],
    eval_dataset=encoded_datasets_EVALution['val'],
    tokenizer=tokenizer, #it is needed the tokenizer that encoded the data for batching
    compute_metrics=compute_metrics #to compute metric of the model in val dataset
)

In [ ]:
trainer.train()

In [ ]:
preds = trainer.predict(test_dataset=encoded_datasets_EVALution['test'])
pred_labels = np.argmax(preds.predictions, axis = 1)

In [ ]:
real_rels = encoded_datasets_EVALution['test'].features['labels'].int2str(encoded_datasets_EVALution['test']['labels'])
pred_rels = encoded_datasets_EVALution['test'].features['labels'].int2str(pred_labels)
report = classification_report(real_rels, pred_rels, digits=3)
report_dict = classification_report(real_rels, pred_rels, digits=3, output_dict=True)
print(report)
print(report_dict)

In [ ]:
dataset = all_datasets_EVALution['test']
file = 'rl_EVALution_16-23_4.txt'
results_to_file(dataset, real_rels, pred_rels, file)

## **ROOT09**

In [ ]:
all_datasets_ROOT09, encoded_datasets_ROOT09 = load_data({'train':'lexical_datasets/ROOT09/train.tsv', 'val':'lexical_datasets/ROOT09/val.tsv', 'test':'lexical_datasets/ROOT09/test.tsv'})
print(all_datasets_ROOT09)
print(encoded_datasets_ROOT09)

In [ ]:
print(all_datasets_ROOT09['train'].features['labels'].names)
num_labels = all_datasets_ROOT09['train'].features['labels'].num_classes
print(f"Number of labels: {num_labels}")

In [ ]:
config = RobertaConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
)
model = RobertaModelWithHeads.from_pretrained(
    model_name,
    config=config,
)

In [ ]:
labels = all_datasets_ROOT09['train']['labels']
rel = all_datasets_ROOT09['train']['rel']
unique_pairs = set(zip(labels, rel))
id2label = dict(unique_pairs)

print(id2label)

In [ ]:
adapter_name = "adapter-ROOT09"

In [ ]:
leave_out = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
print(leave_out)

In [ ]:
# PfeifferConfig + leave_out
adapter_config = AdapterConfig(
    original_ln_before = True,
    original_ln_after = True,
    residual_before_ln = True,
    adapter_residual_before_ln = False,
    ln_before = False,
    ln_after = False,
    mh_adapter = False,
    output_adapter = True,
    non_linearity = "relu",
    reduction_factor = 16,
    leave_out = leave_out
)

In [ ]:
# Add a new adapter
model.add_adapter(adapter_name, config=adapter_config)
# Add a matching classification head
model.add_classification_head(
    adapter_name,
    num_labels=num_labels,
    id2label=id2label
  )
# Activate the adapter
model.train_adapter(adapter_name)

In [ ]:
adapter_layers = model.get_adapter(adapter_name)
print(adapter_layers)

In [ ]:
batch_size = 32
total_epochs = 10

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=total_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=2*batch_size,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    report_to='all',
    load_best_model_at_end=True,     #load the best model at the end of training,
    metric_for_best_model=metric_name,   #use metric_name for validation
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,
    optim='adamw_torch'
)

In [ ]:
trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=encoded_datasets_ROOT09['train'],
    eval_dataset=encoded_datasets_ROOT09['val'],
    tokenizer=tokenizer, #it is needed the tokenizer that encoded the data for batching
    compute_metrics=compute_metrics #to compute metric of the model in val dataset
)

In [ ]:
trainer.train()

In [ ]:
preds = trainer.predict(test_dataset=encoded_datasets_ROOT09['test'])
pred_labels = np.argmax(preds.predictions, axis = 1)

In [ ]:
real_rels = encoded_datasets_ROOT09['test'].features['labels'].int2str(encoded_datasets_ROOT09['test']['labels'])
pred_rels = encoded_datasets_ROOT09['test'].features['labels'].int2str(pred_labels)
report = classification_report(real_rels, pred_rels, digits=3)
report_dict = classification_report(real_rels, pred_rels, digits=3, output_dict=True)
print(report)
print(report_dict)

In [ ]:
dataset = all_datasets_ROOT09['test']
file = 'rl_ROOT09_16-23_4.txt'
results_to_file(dataset, real_rels, pred_rels, file)